In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
import time

# --- 0. Chuẩn bị Thư mục và Thiết bị ---
MODEL_DIR = 'models'
os.makedirs(MODEL_DIR, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Thư mục lưu mô hình đã sẵn sàng: {MODEL_DIR}")
print(f"Sử dụng thiết bị: {device}")

# --- Dữ liệu Giả định (Không Chuẩn hóa) ---
N_SAMPLES = 1000
N_FEATURES = 1
TIMESTEPS = 20

# Tạo dữ liệu giá tiền ảo giả định có biến động lớn
np.random.seed(42)
base_price = np.cumsum(np.random.randn(N_SAMPLES + TIMESTEPS) * 0.1) + 100
# Thêm các 'dị thường' (biến động lớn) ngẫu nhiên
anomaly_indices = np.random.randint(N_SAMPLES // 2, N_SAMPLES, size=10)
base_price[anomaly_indices] += np.random.uniform(5, 10, size=10) # Dị thường lớn
raw_data = base_price.reshape(-1, 1).astype(np.float32)

# --- 1. Hàm Split Data và Tạo Sequence cho LSTM ---
def create_sequences(data, timesteps):
    """Tạo X (sequence) và y (target) cho mô hình LSTM."""
    Xs, ys = [], []
    for i in range(len(data) - timesteps):
        # Lấy một cửa sổ (window) dữ liệu
        X = data[i:(i + timesteps), :]
        # Lấy giá trị tiếp theo làm target cho dự đoán (Model 1)
        y_forecast = data[i + timesteps, :]

        Xs.append(X)
        ys.append(y_forecast)

    return np.array(Xs), np.array(ys)

# Chia Train/Validation/Test (70/15/15)
train_size = int(N_SAMPLES * 0.7)
val_size = int(N_SAMPLES * 0.15)

# Chia tập dữ liệu
train_data = raw_data[:train_size + TIMESTEPS]
val_data = raw_data[train_size:train_size + val_size + TIMESTEPS]

# Tạo sequences
X_train_np, y_train_forecast_np = create_sequences(train_data, TIMESTEPS)
X_val_np, y_val_forecast_np = create_sequences(val_data, TIMESTEPS)

print("\n--- Kích thước Dữ liệu sau khi chia (Không Chuẩn hóa) ---")
print(f"X_train shape: {X_train_np.shape}")
print(f"y_train_forecast shape: {y_train_forecast_np.shape}")


# --- 2. PyTorch Dataset và DataLoader ---

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

BATCH_SIZE = 32

# Model 1 (Dự đoán) DataLoaders
train_dataset_forecast = TimeSeriesDataset(X_train_np, y_train_forecast_np)
train_loader_forecast = DataLoader(train_dataset_forecast, batch_size=BATCH_SIZE, shuffle=False)

val_dataset_forecast = TimeSeriesDataset(X_val_np, y_val_forecast_np)
val_loader_forecast = DataLoader(val_dataset_forecast, batch_size=BATCH_SIZE, shuffle=False)

# Model 2 (Autoencoder) DataLoaders (Input và Target là X)
train_dataset_anomaly = TimeSeriesDataset(X_train_np, X_train_np)
train_loader_anomaly = DataLoader(train_dataset_anomaly, batch_size=BATCH_SIZE, shuffle=False)

val_dataset_anomaly = TimeSeriesDataset(X_val_np, X_val_np)
val_loader_anomaly = DataLoader(val_dataset_anomaly, batch_size=BATCH_SIZE, shuffle=False)


# --- 3. Định nghĩa các Mô hình PyTorch LSTM ---

## Mô hình 1: Dự đoán Chuỗi thời gian
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMForecast, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Output, (h_n, c_n)
        out, _ = self.lstm(x, (h0, c0))
        # Lấy output của bước thời gian cuối cùng
        out = self.fc(out[:, -1, :])
        return out

## Mô hình 2: Phát hiện Bất thường (LSTM Autoencoder)
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMAutoencoder, self).__init__()
        self.timesteps = TIMESTEPS

        # Encoder
        self.encoder = nn.LSTM(input_size, hidden_size, 2, batch_first=True, dropout=0.2)

        # Decoder
        self.decoder = nn.LSTM(hidden_size, input_size, 2, batch_first=True, dropout=0.2)

    def forward(self, x):
        # x shape: (batch_size, timesteps, input_size)

        # 1. Encoder: Học cách nén
        # out_e: output cho từng bước thời gian, (h_n, c_n): hidden/cell state cuối cùng
        _, (hidden_state, cell_state) = self.encoder(x)

        # 2. Decoder Input: Lấy hidden state cuối cùng của Encoder, nhân bản cho mọi bước thời gian
        # Kích thước hidden_state: (num_layers, batch_size, hidden_size)
        # Chúng ta cần hidden state của layer cuối cùng: hidden_state[-1, :, :]
        decoder_input = hidden_state[-1, :, :].unsqueeze(1).repeat(1, self.timesteps, 1)
        # decoder_input shape: (batch_size, timesteps, hidden_size)

        # 3. Decoder: Học cách tái tạo
        # Ở đây chúng ta muốn tái tạo lại chuỗi input ban đầu.
        # Cần điều chỉnh kiến trúc Decoder hoặc sử dụng TimeDistributed.
        # Để đơn giản và phù hợp với cách Keras Autoencoder, ta dùng hidden state làm input cho Decoder

        # Thay vì decoder_input phức tạp, dùng cách đơn giản hơn:
        # Giữ lại hidden state và cell state cuối cùng của Encoder
        # Sử dụng chúng để khởi tạo Decoder

        # Tạo Tensor zero có kích thước (batch_size, timesteps, input_size)
        # Vì decoder sẽ output ra chính input_size
        decoder_input_tensor = torch.zeros(x.size(0), self.timesteps, self.encoder.hidden_size).to(x.device)

        # Giữ lại (h_n, c_n) của encoder
        # Sử dụng output của encoder làm đầu vào cho Decoder.
        # Vì Decoder cần tái tạo lại chuỗi T bước, ta cần một bước trung gian.

        # Cách tiếp cận Autoencoder đơn giản:
        # Output của Encoder (hidden_state[-1]) -> RepeatVector -> Decoder (tái tạo)

        # Input cho Decoder: hidden_state cuối cùng (layer cuối, mọi batch)
        # shape: (batch_size, hidden_size)
        latent_vector = hidden_state[-1]

        # Lặp lại vector nén T lần
        decoder_input = latent_vector.unsqueeze(1).repeat(1, self.timesteps, 1)

        # Decoder: cần LSTM với hidden_size = input_size và output_size = input_size
        # Ta sẽ dùng nn.Linear sau mỗi bước của decoder.

        # Khởi tạo lại Decoder đơn giản hơn:

        out_dec, _ = self.decoder(decoder_input)

        # Áp dụng Linear layer tại mỗi bước thời gian
        # Vì Output của LSTM là (batch_size, timesteps, input_size)
        # Ta cần nn.Linear(input_size, N_FEATURES)

        final_output = nn.Linear(input_size, N_FEATURES).to(x.device)(out_dec)

        return final_output


# --- 4. Hàm Training (Tích hợp Early Stopping) ---

def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs, model_name):
    print(f"\n#################################################################")
    print(f"### BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH: {model_name} ###")
    print(f"#################################################################")

    # Early Stopping Parameters
    patience = 10
    min_delta = 1e-4
    best_val_loss = float('inf')
    epochs_no_improve = 0
    history = {'loss': [], 'val_loss': []}

    for epoch in range(epochs):
        start_time = time.time()

        # --- Training Loop ---
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            output = model(X_batch)

            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # --- Validation Loop ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)

                output = model(X_val)
                val_loss = loss_fn(output, y_val)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        history['loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)

        # In ra loss (theo yêu cầu)
        print(f"Epoch {epoch+1:02d} ({time.time() - start_time:.2f}s): Loss={avg_train_loss:.4f}, Val_Loss={avg_val_loss:.4f}")

        # --- Early Stopping Check ---
        if avg_val_loss < best_val_loss - min_delta:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Lưu checkpoint mô hình tốt nhất
            torch.save(model.state_dict(), os.path.join(MODEL_DIR, f'{model_name}_best_weights.pth'))
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print(f"⚠️ Early stopping tại Epoch {epoch+1:02d}!")
            # Tải lại trọng số tốt nhất trước khi dừng
            model.load_state_dict(torch.load(os.path.join(MODEL_DIR, f'{model_name}_best_weights.pth')))
            break

    return model, history


# --- 5. Huấn luyện Mô hình 1 (Dự đoán) ---
model_forecast = LSTMForecast(
    input_size=N_FEATURES,
    hidden_size=64,
    num_layers=2,
    output_size=N_FEATURES
).to(device)

loss_fn_forecast = nn.MSELoss() # Dùng MSE cho dự đoán giá trị liên tục
optimizer_forecast = torch.optim.Adam(model_forecast.parameters(), lr=1e-3)

model_forecast, history_forecast = train_model(
    model_forecast,
    train_loader_forecast,
    val_loader_forecast,
    loss_fn_forecast,
    optimizer_forecast,
    epochs=100,
    model_name='lstm_forecast'
)

# Lưu mô hình cuối cùng (đã khôi phục trọng số tốt nhất)
torch.save(model_forecast.state_dict(), os.path.join(MODEL_DIR, 'lstm_forecast_model.pth'))
print(f"\n✅ Mô hình 1 đã được lưu tại: {os.path.join(MODEL_DIR, 'lstm_forecast_model.pth')}")


# --- 6. Huấn luyện Mô hình 2 (Bất thường - Autoencoder) ---
# PyTorch Autoencoder phức tạp hơn Keras, ta sẽ đơn giản hóa đầu ra của Decoder để phù hợp.
model_anomaly = LSTMAutoencoder(
    input_size=N_FEATURES,
    hidden_size=64 # hidden_size phải đủ lớn để chứa thông tin nén
).to(device)

loss_fn_anomaly = nn.L1Loss() # Dùng L1Loss (MAE) để đánh giá lỗi tái tạo
optimizer_anomaly = torch.optim.Adam(model_anomaly.parameters(), lr=1e-3)

model_anomaly, history_anomaly = train_model(
    model_anomaly,
    train_loader_anomaly,
    val_loader_anomaly,
    loss_fn_anomaly,
    optimizer_anomaly,
    epochs=100,
    model_name='lstm_anomaly_autoencoder'
)

# Lưu mô hình cuối cùng (đã khôi phục trọng số tốt nhất)
torch.save(model_anomaly.state_dict(), os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth'))
print(f"\n✅ Mô hình 2 đã được lưu tại: {os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth')}")

print("\n🎉 Hoàn tất việc xây dựng, huấn luyện và lưu cả hai mô hình bằng PyTorch!")


# --- Phần bổ sung: Tải Mô hình ---
print("\n--- Phần bổ sung: Tải mô hình đã lưu ---")

# Tải Mô hình 1 (Dự đoán)
loaded_model_forecast = LSTMForecast(N_FEATURES, 64, 2, N_FEATURES).to(device)
loaded_model_forecast.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'lstm_forecast_model.pth')))
loaded_model_forecast.eval()
print("✅ Tải Mô hình 1 thành công.")

# Tải Mô hình 2 (Bất thường)
loaded_model_anomaly = LSTMAutoencoder(N_FEATURES, 64).to(device)
loaded_model_anomaly.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth')))
loaded_model_anomaly.eval()
print("✅ Tải Mô hình 2 thành công.")

In [37]:
import os
import time

import torch
import torch.nn as nn

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader

In [4]:
# Connect google drive
from google.colab import drive
drive.mount("/content/drive" )

Mounted at /content/drive


In [5]:
ROOT_PATH = r"/content/drive/MyDrive/Finance/process"

In [25]:
DATA_PATH = os.path.join(ROOT_PATH, os.listdir(ROOT_PATH)[0])
DATA_PATH

'/content/drive/MyDrive/Finance/process/BTC_1DAY_2024-10-01_to_2025-07-01_with_indicators.csv'

In [27]:
df = pd.read_csv(DATA_PATH)
df = df.set_index("Open Time")


In [28]:
df.head()

,Open,High,Low,Close,Volume,SMA_20,EMA_12,BB_High,BB_Low,RSI_14,MACD,MACD_Signal
Open Time,,,,,,,,,,,,
2024-11-08,75857.89,77199.99,75555.00,76509.78,36521.099583,69932.9505,71896.401049,76087.668111,63778.232889,77.532175,1393.154341,604.405719
2024-11-09,76509.78,76900.00,75714.66,76677.46,16942.079150,70315.2240,72631.948580,77146.832043,63483.615957,77.944505,1671.352739,817.795123
2024-11-10,76677.46,81500.00,76492.00,80370.01,61830.100435,70964.8495,73822.419567,78987.381697,62942.317303,84.958538,2164.830085,1087.202116
2024-11-11,80370.01,89530.54,80216.01,88647.99,82323.665776,72025.9490,76103.276557,83108.094125,60943.803875,91.747320,3187.138889,1507.189470
2024-11-12,88648.00,89940.00,85072.00,87952.01,97299.887911,73090.1170,77926.158625,85950.958533,60229.275467,87.898704,3896.252636,1985.002103


In [29]:
df.describe()

,Open,High,Low,Close,Volume,SMA_20,EMA_12,BB_High,BB_Low,RSI_14,MACD,MACD_Signal
count,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000
mean,95798.944322,97574.281610,93973.738559,95925.306398,27421.958788,94499.630271,95104.159513,101719.951907,87279.308636,54.781431,1074.207264,1075.255294
std,8495.669573,8302.266725,8595.628610,8419.258319,18313.658099,8721.559178,8265.802651,7855.502390,10605.428154,15.998407,2460.719953,2339.878351
min,75857.890000,76900.000000,74508.000000,76322.420000,3282.173520,69932.950500,71896.401049,76087.668111,59431.223270,13.852855,-3518.205705,-3177.898797
25%,88672.292500,91787.415000,86264.497500,89562.092500,14918.436078,86309.193250,88076.940614,98206.074411,79137.596814,43.432302,-816.488278,-803.252701
50%,96576.925000,98101.950000,94953.435000,96600.570000,22238.807157,96735.793250,96782.070619,103001.960335,90783.245479,54.133756,756.225660,876.537712
75%,103270.702500,104985.647500,101243.425000,103339.467500,33389.775120,100172.328000,101671.901800,107632.837630,93536.625224,66.439387,3163.149800,2984.208317
max,111696.220000,111980.000000,109177.370000,111696.210000,109921.729662,106764.890500,107418.612567,112433.116537,102068.143322,91.747320,6693.542852,6003.171432


In [30]:
data = df.drop(columns = ['Volume'])
data.columns

Index(['Open', 'High', 'Low', 'Close', 'SMA_20', 'EMA_12', 'BB_High', 'BB_Low',
       'RSI_14', 'MACD', 'MACD_Signal'],
      dtype='object')

In [31]:
# --- 1. Hàm Split Data và Tạo Sequence cho LSTM ---
def create_sequences(data, timesteps):
    """Tạo X (sequence) và y (target) cho mô hình LSTM."""
    data = np.array(data, dtype=np.float32)
    Xs, ys = [], []
    for i in range(len(data) - timesteps):
        # Lấy một cửa sổ (window) dữ liệu
        X = data[i:(i + timesteps), :]
        # Lấy giá trị tiếp theo làm target cho dự đoán (Model 1)
        y_forecast = data[i + timesteps, :]

        Xs.append(X)
        ys.append(y_forecast)

    return np.array(Xs), np.array(ys)


In [32]:
N_SAMPLES = data.shape[0]
TIMESTEPS = 20
# Chia Train/Validation/Test (70/15/15)
train_size = int(N_SAMPLES * 0.7)
val_size = int(N_SAMPLES * 0.15)
test_size = int(N_SAMPLES * 0.15)

# Chia tập dữ liệu
train_data = data[:train_size + TIMESTEPS]
val_data = data[train_size:train_size + val_size + TIMESTEPS]
test_data = data[train_size + val_size: ]


print (f"Size of train set: {train_data.shape}")
print (f"Size of val set: {val_data.shape}")
print (f"Size of test set: {test_data.shape}")


Size of train set: (185, 11)
Size of val set: (55, 11)
Size of test set: (36, 11)


In [33]:
# Tạo sequences
X_train_np, y_train_forecast_np = create_sequences(train_data, TIMESTEPS)
X_val_np, y_val_forecast_np = create_sequences(val_data, TIMESTEPS)
X_test_np, y_test_forecast_np = create_sequences(test_data, TIMESTEPS)

In [38]:
# --- 2. PyTorch Dataset và DataLoader ---

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

BATCH_SIZE = 32

# Model 1 (Dự đoán) DataLoaders
train_dataset_forecast = TimeSeriesDataset(X_train_np, y_train_forecast_np)
train_loader_forecast = DataLoader(train_dataset_forecast, batch_size=BATCH_SIZE, shuffle=False)

val_dataset_forecast = TimeSeriesDataset(X_val_np, y_val_forecast_np)
val_loader_forecast = DataLoader(val_dataset_forecast, batch_size=BATCH_SIZE, shuffle=False)

# Model 2 (Autoencoder) DataLoaders (Input và Target là X)
train_dataset_anomaly = TimeSeriesDataset(X_train_np, X_train_np)
train_loader_anomaly = DataLoader(train_dataset_anomaly, batch_size=BATCH_SIZE, shuffle=False)

val_dataset_anomaly = TimeSeriesDataset(X_val_np, X_val_np)
val_loader_anomaly = DataLoader(val_dataset_anomaly, batch_size=BATCH_SIZE, shuffle=False)

In [39]:
train_data.iloc[1: 10, : ]

,Open,High,Low,Close,SMA_20,EMA_12,BB_High,BB_Low,RSI_14,MACD,MACD_Signal
Open Time,,,,,,,,,,,
2024-11-09,76509.78,76900.00,75714.66,76677.46,70315.2240,72631.948580,77146.832043,63483.615957,77.944505,1671.352739,817.795123
2024-11-10,76677.46,81500.00,76492.00,80370.01,70964.8495,73822.419567,78987.381697,62942.317303,84.958538,2164.830085,1087.202116
2024-11-11,80370.01,89530.54,80216.01,88647.99,72025.9490,76103.276557,83108.094125,60943.803875,91.747320,3187.138889,1507.189470
2024-11-12,88648.00,89940.00,85072.00,87952.01,73090.1170,77926.158625,85950.958533,60229.275467,87.898704,3896.252636,1985.002103
2024-11-13,87952.00,93265.64,86127.99,90375.20,74198.9630,79841.395760,88966.702730,59431.223270,89.643906,4600.727251,2508.147133
2024-11-14,90375.21,91790.00,86668.21,87325.59,75230.3260,80992.810258,90658.828658,59801.823342,74.121388,4856.962380,2977.910182
2024-11-15,87325.59,91850.00,87073.38,91032.07,76427.2915,82537.311757,92878.203506,59976.379494,79.177714,5298.040018,3441.936149
2024-11-16,91032.08,91779.66,90056.17,90586.92,77555.5525,83775.713025,94661.329030,60449.775970,77.090404,5547.726600,3863.094240
2024-11-17,90587.98,91449.99,88722.00,89855.99,78550.2415,84711.140252,96104.607243,60995.875757,73.423203,5621.820229,4214.839437


In [47]:
# --- 3. Định nghĩa các Mô hình PyTorch LSTM ---

## Mô hình 1: Dự đoán Chuỗi thời gian
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMForecast, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Output, (h_n, c_n)
        out, _ = self.lstm(x, (h0, c0))
        # Lấy output của bước thời gian cuối cùng
        out = self.fc(out[:, -1, :])
        return out




## Mô hình 2: Phát hiện Bất thường (LSTM Autoencoder)
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMAutoencoder, self).__init__()
        self.timesteps = TIMESTEPS

        # Encoder
        self.encoder = nn.LSTM(input_size, hidden_size, 2, batch_first=True, dropout=0.2)

        # Decoder
        self.decoder = nn.LSTM(hidden_size, input_size, 2, batch_first=True, dropout=0.2)

        #
        self.input_size = input_size

    def forward(self, x):
        # x shape: (batch_size, timesteps, input_size)

        # 1. Encoder: Học cách nén
        # out_e: output cho từng bước thời gian, (h_n, c_n): hidden/cell state cuối cùng
        _, (hidden_state, cell_state) = self.encoder(x)

        # 2. Decoder Input: Lấy hidden state cuối cùng của Encoder, nhân bản cho mọi bước thời gian
        # Kích thước hidden_state: (num_layers, batch_size, hidden_size)
        # Chúng ta cần hidden state của layer cuối cùng: hidden_state[-1, :, :]
        decoder_input = hidden_state[-1, :, :].unsqueeze(1).repeat(1, self.timesteps, 1)
        # decoder_input shape: (batch_size, timesteps, hidden_size)

        # 3. Decoder: Học cách tái tạo
        # Ở đây chúng ta muốn tái tạo lại chuỗi input ban đầu.
        # Cần điều chỉnh kiến trúc Decoder hoặc sử dụng TimeDistributed.
        # Để đơn giản và phù hợp với cách Keras Autoencoder, ta dùng hidden state làm input cho Decoder

        # Thay vì decoder_input phức tạp, dùng cách đơn giản hơn:
        # Giữ lại hidden state và cell state cuối cùng của Encoder
        # Sử dụng chúng để khởi tạo Decoder

        # Tạo Tensor zero có kích thước (batch_size, timesteps, input_size)
        # Vì decoder sẽ output ra chính input_size
        decoder_input_tensor = torch.zeros(x.size(0), self.timesteps, self.encoder.hidden_size).to(x.device)

        # Giữ lại (h_n, c_n) của encoder
        # Sử dụng output của encoder làm đầu vào cho Decoder.
        # Vì Decoder cần tái tạo lại chuỗi T bước, ta cần một bước trung gian.

        # Cách tiếp cận Autoencoder đơn giản:
        # Output của Encoder (hidden_state[-1]) -> RepeatVector -> Decoder (tái tạo)

        # Input cho Decoder: hidden_state cuối cùng (layer cuối, mọi batch)
        # shape: (batch_size, hidden_size)
        latent_vector = hidden_state[-1]

        # Lặp lại vector nén T lần
        decoder_input = latent_vector.unsqueeze(1).repeat(1, self.timesteps, 1)

        # Decoder: cần LSTM với hidden_size = input_size và output_size = input_size
        # Ta sẽ dùng nn.Linear sau mỗi bước của decoder.

        # Khởi tạo lại Decoder đơn giản hơn:

        out_dec, _ = self.decoder(decoder_input)

        # Áp dụng Linear layer tại mỗi bước thời gian
        # Vì Output của LSTM là (batch_size, timesteps, input_size)
        # Ta cần nn.Linear(input_size, N_FEATURES)

        final_output = nn.Linear(self.input_size, N_FEATURES).to(x.device)(out_dec)

        return final_output

In [48]:
# --- 4. Hàm Training (Tích hợp Early Stopping) ---


N_FEATURES = len(data.columns)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_DIR = r"/content/drive/MyDrive/Finance/models"
def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs, model_name):
    print(f"\n#################################################################")
    print(f"### BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH: {model_name} ###")
    print(f"#################################################################")

    # Early Stopping Parameters
    patience = 10
    min_delta = 1e-4
    best_val_loss = float('inf')
    epochs_no_improve = 0
    history = {'loss': [], 'val_loss': []}

    for epoch in range(epochs):
        start_time = time.time()

        # --- Training Loop ---
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            output = model(X_batch)

            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # --- Validation Loop ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)

                output = model(X_val)
                val_loss = loss_fn(output, y_val)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        history['loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)

        # In ra loss (theo yêu cầu)
        print(f"Epoch {epoch+1:02d} ({time.time() - start_time:.2f}s): Loss={avg_train_loss:.4f}, Val_Loss={avg_val_loss:.4f}")

        # --- Early Stopping Check ---
        if avg_val_loss < best_val_loss - min_delta:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Lưu checkpoint mô hình tốt nhất
            torch.save(model.state_dict(), os.path.join(MODEL_DIR, f'{model_name}_best_weights.pth'))
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print(f"⚠️ Early stopping tại Epoch {epoch+1:02d}!")
            # Tải lại trọng số tốt nhất trước khi dừng
            model.load_state_dict(torch.load(os.path.join(MODEL_DIR, f'{model_name}_best_weights.pth')))
            break

    return model, history


# --- 5. Huấn luyện Mô hình 1 (Dự đoán) ---
model_forecast = LSTMForecast(
    input_size=N_FEATURES,
    hidden_size=64,
    num_layers=2,
    output_size=N_FEATURES
).to(device)

loss_fn_forecast = nn.MSELoss() # Dùng MSE cho dự đoán giá trị liên tục
optimizer_forecast = torch.optim.Adam(model_forecast.parameters(), lr=1e-3)

model_forecast, history_forecast = train_model(
    model_forecast,
    train_loader_forecast,
    val_loader_forecast,
    loss_fn_forecast,
    optimizer_forecast,
    epochs=100,
    model_name='lstm_forecast'
)

# Lưu mô hình cuối cùng (đã khôi phục trọng số tốt nhất)
torch.save(model_forecast.state_dict(), os.path.join(MODEL_DIR, 'lstm_forecast_model.pth'))
print(f"\n✅ Mô hình 1 đã được lưu tại: {os.path.join(MODEL_DIR, 'lstm_forecast_model.pth')}")


# --- 6. Huấn luyện Mô hình 2 (Bất thường - Autoencoder) ---
# PyTorch Autoencoder phức tạp hơn Keras, ta sẽ đơn giản hóa đầu ra của Decoder để phù hợp.
model_anomaly = LSTMAutoencoder(
    input_size=N_FEATURES,
    hidden_size=64 # hidden_size phải đủ lớn để chứa thông tin nén
).to(device)

loss_fn_anomaly = nn.L1Loss() # Dùng L1Loss (MAE) để đánh giá lỗi tái tạo
optimizer_anomaly = torch.optim.Adam(model_anomaly.parameters(), lr=1e-3)

model_anomaly, history_anomaly = train_model(
    model_anomaly,
    train_loader_anomaly,
    val_loader_anomaly,
    loss_fn_anomaly,
    optimizer_anomaly,
    epochs=100,
    model_name='lstm_anomaly_autoencoder'
)

# Lưu mô hình cuối cùng (đã khôi phục trọng số tốt nhất)
torch.save(model_anomaly.state_dict(), os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth'))
print(f"\n✅ Mô hình 2 đã được lưu tại: {os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth')}")

print("\n🎉 Hoàn tất việc xây dựng, huấn luyện và lưu cả hai mô hình bằng PyTorch!")


# --- Phần bổ sung: Tải Mô hình ---
print("\n--- Phần bổ sung: Tải mô hình đã lưu ---")

# Tải Mô hình 1 (Dự đoán)
loaded_model_forecast = LSTMForecast(N_FEATURES, 64, 2, N_FEATURES).to(device)
loaded_model_forecast.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'lstm_forecast_model.pth')))
loaded_model_forecast.eval()
print("✅ Tải Mô hình 1 thành công.")

# Tải Mô hình 2 (Bất thường)
loaded_model_anomaly = LSTMAutoencoder(N_FEATURES, 64).to(device)
loaded_model_anomaly.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth')))
loaded_model_anomaly.eval()
print("✅ Tải Mô hình 2 thành công.")


#################################################################
### BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH: lstm_forecast ###
#################################################################
Epoch 01 (0.10s): Loss=6478373290.6667, Val_Loss=8103193344.0000
Epoch 02 (0.10s): Loss=6478353408.0000, Val_Loss=8103165184.0000
Epoch 03 (0.10s): Loss=6478320810.6667, Val_Loss=8103110144.0000
Epoch 04 (0.10s): Loss=6478260650.6667, Val_Loss=8103027712.0000
Epoch 05 (0.09s): Loss=6478183338.6667, Val_Loss=8102938880.0000
Epoch 06 (0.10s): Loss=6478100906.6667, Val_Loss=8102840320.0000
Epoch 07 (0.10s): Loss=6478011733.3333, Val_Loss=8102741504.0000
Epoch 08 (0.10s): Loss=6477925290.6667, Val_Loss=8102647552.0000
Epoch 09 (0.11s): Loss=6477843370.6667, Val_Loss=8102557952.0000
Epoch 10 (0.10s): Loss=6477763925.3333, Val_Loss=8102469888.0000
Epoch 11 (0.10s): Loss=6477686698.6667, Val_Loss=8102385152.0000
Epoch 12 (0.10s): Loss=6477611946.6667, Val_Loss=8102302976.0000
Epoch 13 (0.10s): Loss=6477539669.333